In [1]:
### Author: Pongpisit Thanasutives ###
import os
from itertools import combinations
import numpy as np
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, Normalizer
from tqdm import trange
from scipy import io as sio
from scipy.stats import uniform, norm
import pysindy as ps
import pocomc as pc

In [2]:
### algorithm 2 ###
def TopRsq(X_full, y, m, n_tops=25):
    n_feats = X_full.shape[-1]
    r_scores = []
    models = []
    for comb in combinations(range(n_feats), m):
        comb = list(comb)
        active_indices = np.zeros(n_feats)
        active_indices[comb] = 1
        X_sub = X_full[:, comb]
        lr = LinearRegression(fit_intercept=False).fit(X_sub, y)
        R2 = lr.score(X_sub, y)
        r_scores.append(R2)
        models.append(active_indices)
    r_scores = np.array(r_scores)
    r_argsort = np.argsort(r_scores)[::-1][:n_tops]
    r_scores = r_scores[r_argsort]
    models = np.array(models)
    models = models[r_argsort].T
    rating = np.dot(models, r_scores)
    return models, r_scores, rating

In [3]:
### algorithm 3 ###
def comprehensive_search(X_full, y, max_support_size=8, n_tops=None, threshold=0.75):
    X = X_full.copy()
    n_feats = X_full.shape[-1]
    n_tops = int(np.ceil(n_feats/2)) if n_tops is None else n_tops
    ratings = np.zeros((n_feats, max_support_size))
    search = True; support_size = 1
    optimal_indices = None
    i0 = np.arange(n_feats)
    while search and support_size <= max_support_size:
        _, _, rating = TopRsq(X, y, m=support_size, n_tops=n_tops)
        rating = rating/rating.max()
        ratings[:, support_size-1][i0] = rating
        if support_size >= 2:
            i0 = np.nonzero(ratings[:, support_size-1] + ratings[:, support_size-2])[0]
            X = X_full[:, i0]
            i1 = np.where(ratings[:, support_size-1] > threshold)[0]
            i2 = np.where(ratings[:, support_size-2] > threshold)[0]
            if len(i1) == len(i2) and np.all(i1 == i2):
                search = False
                optimal_indices = i1
        support_size += 1
    return optimal_indices, ratings[:, :support_size-1]

In [4]:
n_experiments = 100
n_samples = 10000
n_features = 8
n_informative = 2

threshold = 0.25
max_support_size = 8

success = 0
for i in trange(n_experiments):
    X_train, y_train = make_regression(n_samples=n_samples, n_features=n_features, n_informative=n_informative)
    X_train = StandardScaler().fit_transform(X_train)
    y_train = StandardScaler().fit_transform(y_train.reshape(-1, 1))
    top_models, _, _ = TopRsq(X_train, y_train, m=n_informative)
    true_indices = np.where(top_models[:, 0] > 0)[0]
    est_indices, ratings = comprehensive_search(X_train, y_train, max_support_size=max_support_size, threshold=threshold)
    if est_indices is not None and len(true_indices) == len(est_indices) and np.all(true_indices == est_indices):
        success += 1
        
success/n_experiments

100%|████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:14<00:00,  6.96it/s]


0.91

### PDE ###

In [5]:
data_path = "./Datasets/"
data = sio.loadmat(os.path.join(data_path, "kuramoto_sivishinky.mat"))
print(data.keys())
u_clean = (data['uu']).real; u = u_clean.copy()
x = (data['x'][:, 0]).real
t = (data['tt'][0]).real
dt = t[1]-t[0]; dx = x[2]-x[1]

np.random.seed(0)
noise_type = "gaussian"
noise_lv = float(50)
print("Noise level:", noise_lv)
noise = 0.01*np.abs(noise_lv)*(u.std())*np.random.randn(u.shape[0],u.shape[1])
u = u + noise
u = np.load("./Denoised_data/ks_gaussian50_bm3d.npy")

xt = np.array([x.reshape(-1, 1), t.reshape(1, -1)], dtype=object)
X, T = np.meshgrid(x, t)
XT = np.asarray([X, T]).T

dict_keys(['__header__', '__version__', '__globals__', 'x', 'uu', 'tt'])
Noise level: 50.0


In [6]:
function_library = ps.PolynomialLibrary(degree=2, include_bias=False)

weak_lib = ps.WeakPDELibrary(
    function_library=function_library,
    derivative_order=4,
    spatiotemporal_grid=XT,
    include_bias=True,
    diff_kwargs={"is_uniform":True},
    K=1000
)

X_pre = np.array(weak_lib.fit_transform(np.expand_dims(u, -1)))
y_pre = weak_lib.convert_u_dot_integral(np.expand_dims(u, -1))
y_stan = StandardScaler().fit_transform(y_pre)
N = len(y_pre)

In [7]:
effective_indices, rating = comprehensive_search(X_pre, y_stan, 
                                                 max_support_size=max_support_size,  
                                                 threshold=0.75)
effective_indices, rating

(array([7]),
 array([[0.        , 0.        ],
        [0.        , 0.14676205],
        [0.00322766, 0.        ],
        [0.        , 0.        ],
        [0.        , 0.15307978],
        [0.        , 0.        ],
        [0.        , 0.10141713],
        [1.        , 1.        ],
        [0.00443764, 0.        ],
        [0.        , 0.        ],
        [0.01979575, 0.1635375 ],
        [0.13425153, 0.14912227],
        [0.0059761 , 0.08962063],
        [0.00430323, 0.08952405],
        [0.00331661, 0.10693659]]))

In [8]:
np.nonzero(TopRsq(X_pre, y_pre, m=1)[0][:, 0])[0], \
np.nonzero(TopRsq(X_pre, y_pre, m=2)[0][:, 0])[0], \
np.nonzero(TopRsq(X_pre, y_pre, m=3)[0][:, 0])[0]

(array([7]), array([ 7, 10]), array([4, 6, 7]))

In [9]:
# from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
# efs = EFS(LinearRegression(fit_intercept=False), min_features=3, max_features=3, scoring='r2')
# efs.fit(X_pre, y_pre)
# efs.best_score_, efs.best_idx_, efs.best_feature_names_

### Exat model evidence ###

In [10]:
from scipy.special import loggamma

def log_evidence(X_full, y, effective_indices, v=1):
    y = StandardScaler().fit_transform(y.reshape(-1, 1))
    k = 1 + 1/v
    N = len(y)
    p = len(effective_indices)
    K = X_full[:, effective_indices]
    
    KTy = K.T@y
    yTy = y.T@y
    
    mu = np.linalg.lstsq(K, y, rcond=None)[0]
    Sigma = np.diag(np.ones(p)) * (1 - p/N)/(yTy + mu.T@KTy)[0][0]
    
    Smu = Sigma@mu
    A = K.T@K + Sigma
    A_inv = np.linalg.pinv(A)
    b = KTy + Smu
    xi = (yTy + mu.T@Smu - b.T@(A_inv@b))[0][0]
    
    return N*((np.linalg.slogdet(Sigma)[1] - np.linalg.slogdet(A)[1])/(2*N) - 0.5*np.log(2*np.pi) - \
              (0.5 + k/N)*np.log(xi/2 + 1/v) - (k*np.log(v))/N + (loggamma(N/2 + k) - loggamma(k))/N)

for effective_indices in [[7,], [7, 10], [4, 6, 7]]:
    print(log_evidence(X_pre, y_pre, effective_indices, v=1), 
          log_evidence(X_pre, y_pre, effective_indices, v=1e-2))

-1235.1752294064306 -1239.731391641337
-993.4991272855287 -1030.7564770809772
382.0046233007934 -435.37046735765597


### Bayes factor ###

In [11]:
effective_indices = [7, 10]
X_pre_sub = X_pre[:, effective_indices].copy().T
def log_likelihood(param):
    global X_pre_sub, y_pre, N
    ssr = np.sum(np.abs(param@X_pre_sub - y_pre.flatten())**2, axis=-1)
    def ssr2llf(ssr, nobs):
        nobs2 = nobs / 2.0
        llf = -nobs2 * np.log(2 * np.pi) - nobs2 * np.log(ssr / nobs) - nobs2
        return llf
    return ssr2llf(ssr, N)

n_dim = len(effective_indices)
prior = pc.Prior(n_dim*[norm(0, 1)])
sampler = pc.Sampler(
    prior=prior,
    likelihood=log_likelihood,
    vectorize=True,
)
sampler.run()
simple_samples, simple_weights, _, _  = sampler.posterior()
logz_simple, logz_err_simple = sampler.evidence()

Iter: 32it [00:36,  1.13s/it, beta=1, calls=34304, ESS=3927, logZ=-1.27e+3, logP=-1.27e+3, acc=0.635, steps=3, eff=0.588]   


In [12]:
effective_indices = [4, 7, 10]
X_pre_sub = X_pre[:, effective_indices].copy().T
def log_likelihood(param):
    global X_pre_sub, y_pre, N
    ssr = np.sum(np.abs(param@X_pre_sub - y_pre.flatten())**2, axis=-1)
    def ssr2llf(ssr, nobs):
        nobs2 = nobs / 2.0
        llf = -nobs2 * np.log(2 * np.pi) - nobs2 * np.log(ssr / nobs) - nobs2
        return llf
    return ssr2llf(ssr, N)

n_dim = len(effective_indices)
prior = pc.Prior(n_dim*[norm(0, 1)])
sampler = pc.Sampler(
    prior=prior,
    likelihood=log_likelihood,
    vectorize=True,
)
sampler.run()
extended_samples, extended_weights, _, _  = sampler.posterior()
logz_extended, logz_err_extended = sampler.evidence()

Iter: 36it [00:50,  1.40s/it, beta=1, calls=34560, ESS=3958, logZ=-1.26e+3, logP=-1.25e+3, acc=0.577, steps=7, eff=0.72]   


In [13]:
# Bayes factor of extended to simple model
BF = np.exp(logz_extended-logz_simple)
if BF > 1:
    print('The extended model is more probable than the simple model.')
else:
    print('The simple model is more probable than the extended model.')

The extended model is more probable than the simple model.


In [14]:
np.dot(simple_weights, simple_samples)

array([-0.57864742, -0.04596535])

In [15]:
np.dot(extended_weights, extended_samples)

array([-0.08739292, -0.60465573, -0.03282478])